# Harvard Data ISSN Filter Notebook

This notebook processes the Harvard Data Excel file to keep only those rows where there are values in any of the following columns:
- issn
- issn_l
- issn2
- issn_other_online

The notebook includes additional analysis of the filtered data and maintains all original columns in the output.

In [1]:
import pandas as pd
import numpy as np
import os
import sys
import unicodedata
from datetime import datetime
import logging
import re

# Configuration
INPUT_FILE = 'Output Data/Harvard Data with LCC Subjects.xlsx'
OUTPUT_FILE = 'Output Data/Harvard Data with LCC Subjects and only ISSN.xlsx'

# ISSN-related columns to check
ISSN_COLUMNS = ['issn', 'issn_l', 'issn2', 'issn_other_online']

# Set up Logging

In [2]:
# Configure logging to handle Unicode
def setup_logging():
    """Configure logging with both file and console handlers with Unicode support."""
    # Create logs directory if it doesn't exist
    os.makedirs('logs', exist_ok=True)
    
    # Create formatters for different levels of detail
    brief_formatter = logging.Formatter('%(message)s')
    verbose_formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    
    # Create and configure handlers
    log_filename = f"logs/mods_analysis_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
    
    # Clear any existing handlers
    logger = logging.getLogger()
    for handler in logger.handlers[:]:
        logger.removeHandler(handler)
    
    # File handler with UTF-8 encoding
    file_handler = logging.FileHandler(log_filename, encoding='utf-8')
    file_handler.setLevel(logging.DEBUG)
    file_handler.setFormatter(verbose_formatter)
    
    # Console handler with system encoding
    console_handler = logging.StreamHandler(sys.stdout)
    console_handler.setLevel(logging.INFO)
    console_handler.setFormatter(brief_formatter)
    
    # Configure root logger
    logger.setLevel(logging.DEBUG)
    logger.addHandler(file_handler)
    logger.addHandler(console_handler)
    
    return logger

In [3]:
def normalize_format_string(format_str):
    """Normalize format strings for consistent analysis."""
    if pd.isna(format_str):
        return '[]'
    
    # Remove extra whitespace and normalize quotes
    format_str = format_str.strip().replace("'", '"')
    
    # Ensure string is in list format
    if not format_str.startswith('['):
        format_str = f'[{format_str}]'
    
    # Normalize Unicode characters
    format_str = unicodedata.normalize('NFKD', format_str)
    
    return format_str

def load_excel_file(file_path):
    """Load Excel file and return DataFrame."""
    try:
        logger.info(f"Loading Excel file: {file_path}")
        df = pd.read_excel(file_path)
        logger.info(f"Successfully loaded {len(df)} rows")
        
        # Normalize format strings if format column exists
        if 'format' in df.columns:
            df['format'] = df['format'].fillna('[]').astype(str).apply(normalize_format_string)
        
        return df
    except Exception as e:
        logger.error(f"Error loading file: {str(e)}")
        raise

def validate_columns(df):
    """Validate that at least one ISSN column exists in the DataFrame."""
    existing_columns = [col for col in ISSN_COLUMNS if col in df.columns]
    if not existing_columns:
        raise ValueError(f"None of the required ISSN columns {ISSN_COLUMNS} found in the file")
    logger.info(f"Found ISSN columns: {existing_columns}")
    return existing_columns

def analyze_issn_distribution(df, issn_columns):
    """Analyze the distribution of ISSN values across columns."""
    logger.info("\nISSN Distribution Analysis:")
    
    # Count non-null values in each ISSN column
    for col in issn_columns:
        count = df[col].notna().sum()
        logger.info(f"{col}: {count} values")
    
    # Count rows with multiple ISSN values
    multiple_issns = df[issn_columns].notna().sum(axis=1)
    for i in range(2, len(issn_columns) + 1):
        count = (multiple_issns == i).sum()
        if count > 0:
            logger.info(f"Rows with {i} ISSN values: {count}")

def filter_rows_with_issn(df, issn_columns):
    """Filter DataFrame to keep only rows with values in any ISSN column."""
    # Create a mask for rows that have any ISSN value
    mask = df[issn_columns].notna().any(axis=1)
    
    # Apply the mask and get filtered DataFrame
    filtered_df = df[mask].copy()
    
    # Log the results
    removed_rows = len(df) - len(filtered_df)
    logger.info(f"\nFiltering Results:")
    logger.info(f"Original rows: {len(df)}")
    logger.info(f"Rows with ISSN values: {len(filtered_df)}")
    logger.info(f"Rows removed: {removed_rows}")
    logger.info(f"Percentage of rows kept: {(len(filtered_df)/len(df)*100):.2f}%")
    
    return filtered_df

def analyze_metadata(df, filtered_df):
    """Analyze metadata characteristics of kept vs removed rows."""
    logger.info("\nMetadata Analysis:")
    
    # Analyze type distribution
    if 'type' in df.columns:
        logger.info("\nType distribution in kept rows:")
        type_counts = filtered_df['type'].value_counts()
        for type_name, count in type_counts.items():
            logger.info(f"{type_name}: {count}")
    
    # Analyze format distribution with proper handling of list-like strings
    if 'format' in df.columns:
        logger.info("\nFormat distribution in kept rows (top 20):")
        format_counts = filtered_df['format'].value_counts().head(20)
        
        # Write full format distribution to a separate file
        format_file = 'format_distribution.txt'
        with open(format_file, 'w', encoding='utf-8') as f:
            f.write("Format Distribution (Full):\n\n")
            for format_name, count in filtered_df['format'].value_counts().items():
                f.write(f"{format_name}: {count}\n")
        
        # Log just the top 20 formats
        for format_name, count in format_counts.items():
            logger.info(f"{format_name}: {count}")
        
        logger.info(f"\nFull format distribution written to {format_file}")

def main():
    """Main processing function."""
    try:
        # Load the data
        df = load_excel_file(INPUT_FILE)
        
        # Validate columns
        issn_columns = validate_columns(df)
        
        # Analyze ISSN distribution before filtering
        analyze_issn_distribution(df, issn_columns)
        
        # Filter rows
        filtered_df = filter_rows_with_issn(df, issn_columns)
        
        # Analyze metadata
        analyze_metadata(df, filtered_df)
        
        # Save the filtered DataFrame
        filtered_df.to_excel(OUTPUT_FILE, index=False)
        logger.info(f"\nSaved filtered data to {OUTPUT_FILE}")
        
        # Display sample of filtered data
        logger.info("\nSample of filtered data (first 5 rows):")
        display(filtered_df[['title'] + issn_columns].head())
        
    except Exception as e:
        logger.error(f"Error during processing: {str(e)}")
        raise

In [4]:
# Run the main processing
if __name__ == "__main__":
    logger = setup_logging()
    main()

Loading Excel file: Output Data/Harvard Data with LCC Subjects.xlsx
Successfully loaded 885565 rows
Found ISSN columns: ['issn', 'issn_l', 'issn2', 'issn_other_online']

ISSN Distribution Analysis:
issn: 303561 values
issn_l: 42623 values
issn2: 149276 values
issn_other_online: 7090 values
Rows with 2 ISSN values: 173160
Rows with 3 ISSN values: 3943
Rows with 4 ISSN values: 5963

Filtering Results:
Original rows: 885565
Rows with ISSN values: 303615
Rows removed: 581950
Percentage of rows kept: 34.28%

Metadata Analysis:

Type distribution in kept rows:
serial: 303202
monographic: 228
integrating resource: 133
unknown: 52

Format distribution in kept rows (top 20):
["probably print"]: 79932
["unknown"]: 74411
["probably print", "online resource", "computer", "electronic resource", "remote"]: 62140
["probably print", "online"]: 24349
["probably print", "computer", "online resource"]: 23147
["probably print", "volume", "unmediated"]: 6775
["probably print", "online", "volume", "unmediat

,title,issn,issn_l,issn2,issn_other_online
8,Teḥumin,0333-6883,0333-6883,NaN,NaN
104,"Accessions list, Israel",0041-7750,0041-7750,NaN,NaN
149,Taḳtsiv leʾumi,0334-4738,NaN,NaN,NaN
191,Shenaton hidrologi le-Yisrael,0073-4217,0073-4217,NaN,NaN
198,Duḥot biḳoret ʻal igudim,0792-7932,NaN,NaN,NaN
